In [1]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

from langchain_community.chat_models import ChatOpenAI
from langchain_core.tools import tool
from pydantic.v1 import BaseModel, Field
from typing import Optional

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
import ollama

from semantic_router.utils.function_call import FunctionSchema


/Users/akhilred/Library/Python/3.13/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def read_python_file(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except FileNotFoundError:
        return "Error: File not found."

In [3]:

class codepath(BaseModel):
  path: str = Field(description="code path to execute")

@tool(args_schema = codepath)
def execute_query(path: str) -> str:
  """Returns the result of code path execution"""
  return read_python_file(path)

In [4]:

# defining agent state
class AgentState(TypedDict):
   messages: Annotated[list[AnyMessage], operator.add]

In [5]:


# create the function calling schema for ollama
execute_query_schema = FunctionSchema(read_python_file).to_ollama()
# execute_query_schema["function"]["parameters"]["properties"]["description"] = None
execute_query_schema["function"]["parameters"]["properties"]["description"] = None
execute_query_schema

{'type': 'function',
 'function': {'name': 'read_python_file',
  'description': 'None',
  'parameters': {'type': 'object',
   'properties': {'file_path': {'description': None, 'type': 'object'},
    'description': None},
   'required': []}}}

In [6]:
class codeAgent:
  # initialising the object
  def __init__(self, model, tools, system_prompt = ""):
    self.system_prompt = system_prompt

    # initialising graph with a state 
    graph = StateGraph(AgentState)

    # adding nodes 
    graph.add_node("llm", self.call_llm)
    graph.add_node("function", self.execute_function)
    graph.add_conditional_edges(
      "llm",
      self.exists_function_calling,
      {True: "function", False: END}
    )
    graph.add_edge("function", "llm")

    # setting starting point
    graph.set_entry_point("llm")

    self.graph = graph.compile()
    self.tools = {t.name: t for t in tools}
    self.model = model.bind_tools(tools)

  def call_llm(self, state: AgentState):
    messages = state['messages']
    # adding system prompt if it's defined
    if self.system_prompt:
        messages = [SystemMessage(content=self.system_prompt)] + messages

    # calling LLM
    message = self.model.invoke(messages)

    return {'messages': [message]}
  
  def execute_function(self, state: AgentState):
    tool_calls = state['messages'][-1].tool_calls

    results = []
    for tool in tool_calls:
      # checking whether tool name is correct
      if not tool['name'] in self.tools:
        # returning error to the agent 
        result = "Error: There's no such tool, please, try again" 
      else:
        # getting result from the tool
        result = self.tools[tool['name']].invoke(tool['args'])

      results.append(
        ToolMessage(
          tool_call_id=tool['id'], 
          name=tool['name'], 
          content=str(result)
        )
    )
    return {'messages': results}
  
  def exists_function_calling(self, state: AgentState):
    result = state['messages'][-1]
    return len(result.tool_calls) > 0


# from langchain_ollama import ChatOllama
# model = ChatOllama(model="llama3.2:1b")
# # model = ChatOllama(model="codellama:latest")

# system prompt
# prompt = '''You are a senior expert in reviewing python code. 
# So, you can help the team to review the code and provide feedback. 
# You are very accurate and take into account all the nuances in code.
# Your goal is to provide the detailed documentation for any security issues in the code that will help users.'''

prompt = '''You are a senior expert in reviewing code for observability. The best one that exists.
Your goal is to analyze the code on the following questions 
1. Are there actionable alerts identified for the feature? Are there Runbooks for the actionable alerts? Do we have TSGs attached to the alert?
2. Add metrics to monitor dependencies and exception handling on components, infrastructure and features so that SRE can create alerts to reduce TTD?
3. Are there CorrelationIDs established in logs to derive error lineage across various components?
4. Can the feature/service support changing log levels for troubleshooting purposes?
5. Are there critical log lines that we need to get alerted upon?
Provide response in the format as follows: {question: response}
'''

import os
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel

model = AzureAIChatCompletionsModel(
    endpoint= os.getenv('azureaifoundry4oendpoint'),
    credential=os.getenv('azureaifoundry4okey'),
    model="gpt-4o-mini",
)

doc_agent = codeAgent(model, [execute_query], system_prompt=prompt)

messages = [HumanMessage(content="the code is in the path '/Users/akhilred/Desktop/Billing Pyton Script/Billing_Usage_Extraction.py'. Analyze the code for observability requirements mentioned in the prompt")]
result = doc_agent.graph.invoke({"messages": messages})

2025-03-26 15:12:55 - azure.core.pipeline.policies.http_logging_policy - INFO - _universal.py:509 - on_request() - Request URL: 'https://agenticaieastu6274613629.openai.azure.com/openai/deployments/gpt-4o-mini/info?api-version=REDACTED'
Request method: 'GET'
Request headers:
    'Accept': 'application/json'
    'x-ms-client-request-id': '15d349cc-0a87-11f0-81d4-52304e248077'
    'api-key': 'REDACTED'
    'User-Agent': 'langchain-azure-ai azsdk-python-ai-inference/1.0.0b9 Python/3.13.0 (macOS-15.3.2-arm64-arm-64bit-Mach-O)'
    'Authorization': 'REDACTED'
No body was attached to the request
2025-03-26 15:12:55 - azure.core.pipeline.policies.http_logging_policy - INFO - _universal.py:545 - on_response() - Response status: 404
Response headers:
    'Content-Length': '56'
    'Content-Type': 'application/json'
    'apim-request-id': 'REDACTED'
    'Strict-Transport-Security': 'REDACTED'
    'x-content-type-options': 'REDACTED'
    'Date': 'Wed, 26 Mar 2025 21:12:54 GMT'
2025-03-26 15:12:55

In [7]:
print(result['messages'][3].content)

Here is the analysis of the provided code regarding the observability requirements:

1. **Are there actionable alerts identified for the feature? Are there Runbooks for the actionable alerts? Do we have TSGs attached to the alert?**
   - **Response:** The code does not include any built-in alerts, runbooks, or TSG (trouble shooting guides) frameworks. There is no logic in place to generate alerts related to failure conditions, such as unsuccessful data extraction or exceptions. It is recommended to implement appropriate monitoring and alerting mechanisms based on expected failure points.

2. **Add metrics to monitor dependencies and exception handling on components, infrastructure and features so that SRE can create alerts to reduce TTD?**
   - **Response:** The code currently lacks metrics that would allow monitoring of dependencies or exception handling. It only catches `KustoServiceError` exceptions in the data extraction methods. Adding logging for successes, failures, execution ti